# CTGAN for Synthetic Data Generation using MIMIC-III

## Steps:
1. Load & Preprocess MIMIC-III Data

2. Train CTGAN on MIMIC-III Data

3. Generate & Evaluate Synthetic Data

Install and import libraries

In [1]:
!pip install --upgrade ctgan sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.1/156.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Dataframe
import pandas as pd

# MetaData
from sdv.metadata import Metadata
# Model
from sdv.single_table import CTGANSynthesizer

# Evaluation
from sdv.evaluation.single_table import run_diagnostic
from sdv.evaluation.single_table import evaluate_quality

## Step 1: Load & Preprocess MIMIC-III Data

*Note: Make sure that the files are uploaded in the google colab. - ADMISSIONS.csv.gz, PATIENTS.csv.gz, ICUSTAYS.csv.gz*

In [3]:
# Load MIMIC-III CSV.GZ files
admissions = pd.read_csv("ADMISSIONS.csv.gz", compression="gzip")
patients = pd.read_csv("PATIENTS.csv.gz", compression="gzip")
icustays = pd.read_csv("ICUSTAYS.csv.gz", compression="gzip")

# Sample subset for Assignment
admissions = admissions.sample(2000)
patients = patients.sample(2000)
icustays = icustays.sample(2000)

# Select relevant columns
admissions = admissions[["SUBJECT_ID", "HADM_ID", "ADMISSION_TYPE", "INSURANCE", "ETHNICITY", "ADMITTIME"]]
patients = patients[["SUBJECT_ID", "DOB"]]
icustays = icustays[["HADM_ID", "LOS"]]

# Convert date columns to datetime
admissions["ADMITTIME"] = pd.to_datetime(admissions["ADMITTIME"])
patients["DOB"] = pd.to_datetime(patients["DOB"])

# Compute age at admission
admissions = admissions.merge(patients, on="SUBJECT_ID", how="left")
admissions["AGE"] = admissions["ADMITTIME"].dt.year - admissions["DOB"].dt.year

# Merge with ICU stays to get Length of Stay (LOS)
df = admissions.merge(icustays, on="HADM_ID", how="left").rename(columns={"LOS": "ICU_LOS"})

# Keep only relevant columns
df = df[["AGE", "ICU_LOS", "ADMISSION_TYPE", "INSURANCE", "ETHNICITY"]]

# Handle missing values
# df.fillna("Unknown", inplace=True)
for col in df.columns:
    if df[col].dtype == "object":  # Categorical columns
        df[col].fillna("Unknown", inplace=True)
    else:  # Numerical columns (e.g., ICU_LOS)
        df[col].fillna(-1, inplace=True)  # Use -1 as a placeholder for missing values

# Convert categorical variables to strings
categorical_columns = ["ADMISSION_TYPE", "INSURANCE", "ETHNICITY"]
df[categorical_columns] = df[categorical_columns].astype(str)

# Save processed data
df.to_csv("mimic_ctgan_data.csv", index=False)

print("MIMIC-III data preprocessed and saved for CTGAN training!")
print(df.head())

MIMIC-III data preprocessed and saved for CTGAN training!
   AGE  ICU_LOS ADMISSION_TYPE INSURANCE ETHNICITY
0 -1.0     -1.0      EMERGENCY   Private     WHITE
1 -1.0     -1.0      EMERGENCY   Private     WHITE
2 -1.0     -1.0      EMERGENCY   Private     WHITE
3 -1.0     -1.0        NEWBORN   Private     WHITE
4 -1.0     -1.0      EMERGENCY  Medicare     WHITE


<ipython-input-3-a8ccfbe62034>:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(-1, inplace=True)  # Use -1 as a placeholder for missing values
<ipython-input-3-a8ccfbe62034>:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(v

## Step 2: Train CTGAN on MIMIC-III Data

*Note: the below code triggers GAN training and may take some time. Try changing the runtime to GPU and reduce the number of epochs if it is taking too long to run.*

In [4]:
# Load preprocessed data
df = pd.read_csv("/content/mimic_ctgan_data.csv")

# Define metadata (categorical vs. numerical)
metadata = Metadata.detect_from_dataframe(data=df, table_name='ctgan_data')

# Initialize and train CTGAN
ctgan = CTGANSynthesizer(metadata, enforce_rounding=False, epochs=70, verbose=True)
print("Training CTGAN...")
ctgan.fit(df)

# Save trained model
import pickle
with open("ctgan_model.pkl", "wb") as f:
    pickle.dump(ctgan, f)

print("CTGAN training complete! Model saved.")

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Training CTGAN...


Gen. (0.43) | Discrim. (-0.07): 100%|██████████| 70/70 [00:07<00:00,  9.28it/s]

CTGAN training complete! Model saved.


In [5]:
ctgan.get_loss_values_plot()

## Step 3: Generate & Evaluate Synthetic Data

### Generate synthetic data

In [6]:
# Generate synthetic data
synthetic_data = ctgan.sample(num_rows=1000)
synthetic_data.to_csv("synthetic_mimic_data.csv", index=False)
print("Synthetic data generated and saved!")

Synthetic data generated and saved!


In [7]:
print(synthetic_data.info)
synthetic_data.head()

<bound method DataFrame.info of           AGE   ICU_LOS ADMISSION_TYPE   INSURANCE  \
0   -1.000000 -0.999069      EMERGENCY    Medicare   
1   -0.406962 -0.980822       ELECTIVE     Private   
2   -0.605185 -0.971455      EMERGENCY  Government   
3   -0.439996 -0.988964      EMERGENCY    Medicaid   
4   -1.000000 -1.000000      EMERGENCY  Government   
..        ...       ...            ...         ...   
995 -0.641055 -0.883165      EMERGENCY    Medicaid   
996 -1.000000 -1.000000      EMERGENCY    Medicare   
997 -0.842612 -0.919330      EMERGENCY    Self Pay   
998 -1.000000 -0.961767      EMERGENCY     Private   
999  0.356325 -0.982706      EMERGENCY     Private   

                          ETHNICITY  
0                             WHITE  
1                             WHITE  
2                             WHITE  
3             UNKNOWN/NOT SPECIFIED  
4                             WHITE  
..                              ...  
995           UNKNOWN/NOT SPECIFIED  
996            

,AGE,ICU_LOS,ADMISSION_TYPE,INSURANCE,ETHNICITY
0,-1.000000,-0.999069,EMERGENCY,Medicare,WHITE
1,-0.406962,-0.980822,ELECTIVE,Private,WHITE
2,-0.605185,-0.971455,EMERGENCY,Government,WHITE
3,-0.439996,-0.988964,EMERGENCY,Medicaid,UNKNOWN/NOT SPECIFIED
4,-1.000000,-1.000000,EMERGENCY,Government,WHITE


The above generated records are synthetically generated. Enabling us to overcome data privacy issues.

### Evaluate data

In [8]:
def evaluate_synthetic_data(real_data, synthetic_data, metadata):
    """Evaluate the quality of generated synthetic data."""
    diagnostic = run_diagnostic(real_data=real_data, synthetic_data=synthetic_data, metadata=metadata)
    quality_report = evaluate_quality(real_data, synthetic_data, metadata)
    return diagnostic, quality_report

In [9]:
diagnostic, quality_report = evaluate_synthetic_data(df, synthetic_data, metadata)

Generating report ...

(1/2) Evaluating Data Validity: |██████████| 5/5 [00:00<00:00, 573.49it/s]|
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 433.25it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 5/5 [00:00<00:00, 103.93it/s]|
Column Shapes Score: 66.72%

(2/2) Evaluating Column Pair Trends: |██████████| 10/10 [00:00<00:00, 146.58it/s]|
Column Pair Trends Score: 79.18%

Overall Score (Average): 72.95%



In [10]:
quality_report.get_details('Column Shapes')

,Column,Metric,Score
0,AGE,KSComplement,0.4790
1,ICU_LOS,KSComplement,0.3935
2,ADMISSION_TYPE,TVComplement,0.9035
3,INSURANCE,TVComplement,0.8345
4,ETHNICITY,TVComplement,0.7255


In [11]:
# Visualization
from sdv.evaluation.single_table import get_column_plot

def plot_column_distributions(real_data, synthetic_data, metadata, column_names):
    """Plot distributions of specified columns between real and synthetic data."""
    for col in column_names:
        fig = get_column_plot(real_data=real_data, synthetic_data=synthetic_data, metadata=metadata, column_name=col)
        fig.show()

plot_column_distributions(df, synthetic_data, metadata, df.columns)

Note: We observe that the synthetic data generated closely resembles the distribution of real MIMIC dataset.

## Conclusion

To overcome the challenge of data privacy with healthcare datasets like MIMIC III, we can synthetically create synthetic dataset using CTGAN and evaluate it. This new synthetic dataset can then be freely used, overcoming some challenges with healthcare data.